<h3 style='color:orange'> Importação das Bibliotecas </h3>

In [1]:
import connections
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
connection = connections.get_db()

## Obtenção dos dados

In [10]:
#https://www.kaggle.com/datasets/felipejardimf/hotel-reviews-hostelworld
df_raw = pd.read_csv('hostel_detail_data.csv',sep=';')

In [11]:
df_raw.columns

Index(['id_hostel', 'name', 'currency', 'type', 'maxNumberOfGuestsPerBooking',
       'isFeatured', 'latestCheckOut', 'rating', 'totalRatings', 'starRating',
       'facilities', 'description', 'latitude', 'longitude', 'address1',
       'address2', 'directions', 'depositPercentage', 'checkIn.startsAt',
       'checkIn.endsAt', 'city.id', 'city.name', 'city.country',
       'city.idCountry', 'region.id', 'region.name', 'payments.payNow',
       'cancellationPolicy.depositOnly.description',
       'cancellationPolicy.depositOnly.numberOfDays',
       'cancellationPolicy.nonRefundable.description',
       'cancellationPolicy.freeCancellation.description',
       'freeCancellation.isAvailable', 'freeCancellation.daysInAdvance',
       'groupInformation.groupMaxPax', 'taxInfo.included', 'taxInfo.taxes'],
      dtype='object')

In [12]:
# Remover colunas que não serão usadas
colunas_drop = ['rating',
                'directions',
                'groupInformation.groupMaxPax',
                'cancellationPolicy.nonRefundable.description',
                'cancellationPolicy.nonRefundable.description',
                'address2',
                'cancellationPolicy.freeCancellation.description',
                'cancellationPolicy.depositOnly.numberOfDays',
                'taxInfo.taxes',
                'maxNumberOfGuestsPerBooking',
                'isFeatured',
                'city.id',
                'city.idCountry',
                'region.id',
                'payments.payNow',
                'cancellationPolicy.depositOnly.description'
               ]

df = df_raw.drop(columns = colunas_drop)

In [13]:
# Renomear as colunas
df.rename(columns = {
        'latestCheckOut':   'last_checkout',
        'totalRatings':     'qtd_rating',
        'starRating':       'min_rating',
        'address1':         'address',
        'depositPercentage':'deposit_percentage',
        'checkIn.startsAt': 'checkin_start',
        'checkIn.endsAt':   'checkin_end',
        'city.name':        'city',
        'city.country':     'country',
        'region.name':      'region',
        'freeCancellation.isAvailable': 'free_cancelation_is_available',
        'freeCancellation.daysInAdvance': 'free_cancelation_days_in_advance',
        'taxInfo.included': 'tax_is_included'
    }, inplace = True
)

In [14]:
df['facilities'] = df['facilities'].apply(eval)

<h3 style='color:orange'> Hostels </h3>

In [19]:
# Fazer uma cópia para poder remover a coluna de Facilities
df_insert = df.copy()

# Remover coluna não usada
df_insert.drop(columns = ['facilities'], inplace = True)

In [20]:
#https://www.kaggle.com/datasets/hserdaraltan/countries-by-continent 
continentes = pd.read_csv('countryContinent.csv')

In [21]:
# Obter os continentes dos Países
df_insert = pd.merge(
    left = df_insert,
    right = continentes,
    left_on = 'country',
    right_on = 'Country',
    how = 'left'
)

In [22]:
# Mover a posição dele
df_insert.insert(17,'continent', df_insert['Continent'])

# Remover as antigas colunas
df_insert.drop(columns = ['Continent','Country'], inplace = True)

In [23]:
# Adicionar a URL 
df_insert['url'] = df_insert['id_hostel'].apply(lambda id_hostel: f"https://www.hostelworld.com/st/hostels/p/{id_hostel}/{id_hostel}")

In [24]:
df_insert.columns

Index(['id_hostel', 'name', 'currency', 'type', 'last_checkout', 'qtd_rating',
       'min_rating', 'description', 'latitude', 'longitude', 'address',
       'deposit_percentage', 'checkin_start', 'checkin_end', 'city', 'country',
       'region', 'continent', 'free_cancelation_is_available',
       'free_cancelation_days_in_advance', 'tax_is_included', 'url'],
      dtype='object')

In [25]:
df_insert

,id_hostel,name,currency,type,last_checkout,qtd_rating,min_rating,description,latitude,longitude,address,deposit_percentage,checkin_start,checkin_end,city,country,region,continent,free_cancelation_is_available,free_cancelation_days_in_advance,tax_is_included,url
0,177051,NattyDog,EUR,HOSTEL,NaN,10,0,NattyDog Hostel offers accommodation in city c...,47.485435,19.064035,Raday street 31/j 2/11,15,13,23,Budapest,Hungary,Dunakeszi,Europe,True,3.0,True,https://www.hostelworld.com/st/hostels/p/17705...
1,177048,Hostel MAK Sarajevo,EUR,GUESTHOUSE,NaN,1,0,Hostel Mak offers accommodation in Sarajevo. H...,43.858985,18.424663,"Ferhadija - Stakleni grad, Kvadrant 11",15,14,18,Sarajevo,Bosnia And Herzegovina,NaN,NaN,True,4.0,True,https://www.hostelworld.com/st/hostels/p/17704...
2,177041,StarVillas Apartments,EUR,APARTMENT,NaN,5,0,StarVillas are newly built apartments and mais...,38.301725,20.596168,Agia Efimia,15,0,23,Kefalonia,Greece,Argostolion,Europe,True,4.0,True,https://www.hostelworld.com/st/hostels/p/17704...
3,177037,Andrews Apartments,EUR,APARTMENT,NaN,0,0,Nafplio has always been an ideal destination f...,37.569876,22.828744,Aria Nafplio,15,14,22,Nafplio,Greece,Nafplion,Europe,True,4.0,True,https://www.hostelworld.com/st/hostels/p/17703...
4,177035,Hostal El Parque,USD,GUESTHOUSE,NaN,4,0,Hostal El Parque offers a warm atmosphere surr...,-0.185545,-78.344753,Calle 29 de Abril Oe1-125 y 24 de Septiembre,15,12,23,Tababela,Ecuador,Quito,South America,True,4.0,True,https://www.hostelworld.com/st/hostels/p/17703...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41637,300007,Hosteleski,TRY,HOSTEL,12:00,2,0,Hosteleski was founded in 2012 and is the firs...,39.775533,30.514891,"İstiklal Mahallesi, Yalaman Sokak, No.12",15,14,24,Eskisehir,Turkey,NaN,Europe,True,3.0,True,https://www.hostelworld.com/st/hostels/p/30000...
41638,300005,Shaka Seagypsy Langkawi,MYR,GUESTHOUSE,12:00,2,0,"Dear guests, \r\n\r\nThank you for choosing SH...",6.362430,99.712268,"78, Jalan Pantai Kok",15,13,23,Langkawi,Malaysia,Kedah,Asia,True,3.0,False,https://www.hostelworld.com/st/hostels/p/30000...
41639,300004,Chakana Hostel,ARS,HOSTEL,10:00,6,0,"Chakana Hostel has four shared rooms, with top...",-28.465777,-65.778387,Prado 673,15,13,23,San Fernando del Valle de Catamarca,Argentina,NaN,South America,True,2.0,True,https://www.hostelworld.com/st/hostels/p/30000...
41640,300000,Hakuna Matata Eco-Hostal,USD,HOTEL,NaN,0,0,Hakuna Matata Eco-Hostal is an eco-friendly ho...,10.228516,-75.606121,"Playa Blanca, Isla Baru, Sector Motica",15,14,23,Cartagena,Colombia,Bolivar,South America,True,2.0,True,https://www.hostelworld.com/st/hostels/p/30000...


In [26]:
df_insert.to_sql('hostelworld_hostel', connection, if_exists='replace',index = False, method = 'multi')

41642

## Normalizações

<h3 style='color:orange'> Categoria de Facilidade </h3>

In [9]:
# Explodir o DataFrame para obtenção de todas as Categorias
df_exploded = df[['id_hostel','facilities']].explode('facilities').reset_index(drop = True)

# Usar o Json_Normalize para criar o DataFrame
df_category_raw = pd.json_normalize(df_exploded['facilities'])

# Adicionar o Id do Hostel
df_category_raw.insert(0, 'id_hostel', df_exploded['id_hostel'])

In [10]:
# Criar o DataFrame somente com as Categorias Existentes
df_category = df_category_raw[['numericId','name','id']].drop_duplicates()

# Remover registros Nulos
df_category = df_category.dropna()

# Ajustaro  tipo de dados para INT
df_category['numericId'] = df_category['numericId'].astype(int)

# Renomear as colunas
df_category.rename(columns = {
    'numericId':'id_category',
    'name':'category',
}, inplace = True)

# Remover colunas desnecessarias
df_category.drop(columns = ['id'], inplace = True)

#### Inserir no Banco de dados

In [12]:
df_category.to_sql('hostelworld_category_facilities', connection, index = False, if_exists = 'append')

5

<h3 style='color:orange'> Facilidades </h3>

In [13]:
# Explodir em mais uma camada para obter as facilidades das categorias
df_exploded = df_category_raw[['id_hostel','numericId','facilities']].explode('facilities').reset_index(drop = True)

# Criar o Novo DataFrame contendo as Facilidades
df_facilities_raw  = pd.json_normalize(df_exploded['facilities'])

In [14]:
# Remover os Registros Nulos
df_facilities_raw = df_facilities_raw.dropna()

# Inserir as colunas de Identificação do Hostel e da Categoria da Facilidade
df_facilities_raw.insert(0,'id_category', df_exploded['numericId'])
df_facilities_raw.insert(0,'id_hostel', df_exploded['id_hostel'])

In [15]:
# Criar o DataFrame das Facilidades que existem
df_facilities = df_facilities_raw[['numericId','name','id']].drop_duplicates()

# Remover os Nulos
df_facilities = df_facilities.dropna()

# Ajustar os tipos de dados
df_facilities['numericId'] = df_facilities['numericId'].astype(int)

# Renomear as colunas
df_facilities.rename(columns = {
    'numericId':'id_facility',
    'name':'facility'
}, inplace = True)

### Inserir a tabela de Facilidades no Banco

In [16]:
df_facilities

,id_facility,facility,id
0,80,Free City Maps,FREECITYMAPS
1,90,Free WiFi,FREEWIFI
2,87,Towels Included,TOWELSINCLUDED
3,37,Breakfast Not Included,BREAKFASTNOTINCLUDED
4,27,Elevator,ELEVATOR
...,...,...,...
3075,113,Jobs Board,JOBSBOARD
3586,144,Sauna,SAUNA
7552,143,Golf Course,GOLFCOURSE
11730,302,UKRAINEREFUGEES,UKRAINEREFUGEES


In [17]:
df_facilities.to_sql('hostelworld_facilities', connection, index = False, if_exists = 'append')

103

<h3 style='color:orange'> Tabela Fato </h3>

In [18]:
# Renomear as colunas
df_facilities_raw.rename(columns = {
    'numericId': 'id_facility'
}, inplace = True)

# Ajustar os tipos de dados
df_facilities_raw['id_category'] = df_facilities_raw['id_category'].astype(int)
df_facilities_raw['id_facility'] = df_facilities_raw['id_facility'].astype(int)

# Remover colunas não usadas
df_facilities_raw.drop(columns = ['name','id'], inplace = True)

### Inserir a tabela Fato no Banco

In [19]:
df_facilities_raw

,id_hostel,id_category,id_facility
0,177051,1,80
1,177051,1,90
2,177051,1,87
3,177051,2,37
4,177051,2,27
...,...,...,...
887910,300001,3,4
887911,300001,3,15
887912,300001,3,13
887913,300001,4,125


In [21]:
df_facilities_raw.to_sql('hostelworld_hostel_facilities', connection, index = False, if_exists = 'append', method = 'multi')

887375